In [1]:
import numpy as np
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from tqdm import tqdm

In [2]:
cz, dm, ra, dec = np.loadtxt('data/allCF4_individual.txt', skiprows=5, unpack=True, usecols=[3, 4, -9,-8])

In [79]:
f = open('data/matching.txt', 'w')
f.write('ra_cf4 dec_cf4, ra_sdss, dec_sdss r objid\n')
for r, d in tqdm(zip(ra, dec), total=len(ra)):
    # Define the coordinate object
    coord = coords.SkyCoord(r, d, unit="deg", frame='icrs')
    
    try:
        # Query for spectra within 2 arcseconds of the coordinate
        xid = SDSS.query_region(coord, spectro=True, radius='1arcsec', photoobj_fields=['r', 'objid'],
                            specobj_fields=['ra', 'dec', 'plate', 'mjd', 'fiberID'], timeout=300)
    
        f.write('%f %f %f %f %f %d %d %d %d\n'%(r, d, float(xid['ra'][0]), float(xid['dec'][0]),
                                     float(xid['r'][0]), int(xid['objid'][0]), int(xid['plate'][0]),
                                     int(xid['mjd'][0]), int(xid['fiberID'][0])))
    except:
        f.write(f'{r} {d} -999 -9999 -9 -9 -9 -9 -9\n')
        
f.close()

In [41]:
matched_ra, matched_dec, r, plate, mjd, fiberid = np.loadtxt('data/matching.txt', skiprows=1, unpack=True, usecols=[2,3,4,6,7,8])
import matplotlib.pyplot as plt
from scl_plot import scl_plt
from specutils import Spectrum1D
from astropy.constants import c
c = c.to_value('km/s')
import os

ibright = np.where((r>-9)&(r<11.81))[0]

os.system('rm -fr data/spectrum/*')
dist = open(f'data/dist.txt', 'w')
dist.write('spec dist\n')
for n,i in enumerate(ibright):
    spec_hdu = SDSS.get_spectra(plate=plate[i], mjd=mjd[i], fiberID=fiberid[i], data_release=14)
    spec = Spectrum1D.read(spec_hdu[0], format="SDSS-III/IV spec")
    lamb, flux = spec.spectral_axis.value, spec.flux.value
    spec_txt = open(f'data/spectrum/spec{n}.txt', 'w')
    spec_txt.write('lamb flux')
    for l, f in zip(lamb, flux):
        spec_txt.write(f'{l} {f}\n')
    spec_txt.close()
    
    d = 10**(dm[i]*0.2+1)/1e+6
    dist.write(f'{n} {d}\n')
    
dist.close()

In [38]:
def init_measure_redshift():
    file = open('measure_redshift.txt', 'w')
    file.write('spec z')
    for n in range(len(ibright)):
        file.write(f'{n:d} {0.:5f}\n')
    file.close()
init_measure_redshift()

In [209]:
from astroquery.sdss import SDSS
from astropy.table import Table
import pandas as pd

# Query for all objects with RA, DEC, and redshift information
# This example shows how to query a specific region for demonstration purposes
query = """
    SELECT ra, dec, z
    FROM specObj
    WHERE z > -0.5 AND z < 1
"""

# Perform the query
results = SDSS.query_sql(query, read_timeout=120)
pd_sdss = results.to_pandas()
pd_sdss.to_csv('data/z_sdss', sep=' ', index=False)